In [6]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


## Importing the libraries

In [23]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers, models
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Load and Prepare Data

In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print("shape of picture:")
print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

170498071/170498071 [==============================] - 2s 0us/step
shape of picture:
x_train: (50000, 32, 32, 3)
x_test: (10000, 32, 32, 3)
y_train: (50000, 1)
y_test: (10000, 1)



## Prepare Data 

In [9]:
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [10]:
y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)

## Train your model

In [11]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(32, 32, 3)))
    model.add(layers.Dense(units=hp.Int('units',
                                         min_value=32,
                                         max_value=512,
                                         step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='cifar10')

In [14]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 5 Complete [00h 06m 33s]
val_accuracy: 0.42733333508173627

Best val_accuracy So Far: 0.43271111448605853
Total elapsed time: 00h 24m 57s


In [15]:
best_model = tuner.get_best_models(num_models=1)[0]

In [16]:
test_loss, test_acc = best_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.5586 - accuracy: 0.4449


In [17]:
y_pred = best_model.predict(x_test)

313/313 [==============================] - 2s 5ms/step


In [21]:
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')

In [24]:
roc_auc = roc_auc_score(y_test, y_pred, average='macro', multi_class='ovo')

In [25]:
print("Test Accuracy:", test_acc)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

Test Accuracy: 0.4449000060558319
F1 Score: 0.43683983851015107
ROC AUC Score: 0.854163061111111


In [26]:
tuner.results_summary()


Results summary
Results in my_dir/cifar10
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
units: 448
Score: 0.43271111448605853

Trial 4 summary
Hyperparameters:
units: 384
Score: 0.42733333508173627

Trial 2 summary
Hyperparameters:
units: 224
Score: 0.4159555534521739

Trial 0 summary
Hyperparameters:
units: 192
Score: 0.4065333306789398

Trial 3 summary
Hyperparameters:
units: 32
Score: 0.13062222301959991


In [27]:
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model

In [28]:
def build_wide_and_deep_model(hp):
    input_layer = Input(shape=(32, 32, 3))
    flattened_input = layers.Flatten()(input_layer)


    wide = layers.Dense(units=hp.Int('wide_units', min_value=32, max_value=512, step=32), activation='relu')(flattened_input)


    deep = layers.Dense(units=hp.Int('deep_units', min_value=32, max_value=512, step=32), activation='relu')(flattened_input)
    deep = layers.Dense(units=hp.Int('deep_units', min_value=32, max_value=512, step=32), activation='relu')(deep)


    concatenated = Concatenate()([wide, deep])


    output = layers.Dense(10, activation='softmax')(concatenated)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [29]:
tuner_wide_and_deep = RandomSearch(
    build_wide_and_deep_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='cifar10_wide_and_deep')

In [30]:
tuner_wide_and_deep.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 5 Complete [00h 16m 16s]
val_accuracy: 0.42888887723286945

Best val_accuracy So Far: 0.4328888952732086
Total elapsed time: 00h 55m 55s


In [31]:
best_wide_and_deep_model = tuner_wide_and_deep.get_best_models(num_models=1)[0]


In [32]:
test_loss_wide_and_deep, test_acc_wide_and_deep = best_wide_and_deep_model.evaluate(x_test, y_test)



313/313 [==============================] - 3s 9ms/step - loss: 1.5785 - accuracy: 0.4450


In [33]:
y_pred_wide_and_deep = best_wide_and_deep_model.predict(x_test)

313/313 [==============================] - 3s 9ms/step


In [34]:
f1_wide_and_deep = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred_wide_and_deep, axis=1), average='weighted')

In [35]:
roc_auc_wide_and_deep = roc_auc_score(y_test, y_pred_wide_and_deep, average='macro', multi_class='ovo')

In [36]:
print("Wide and Deep Model:")
print("Test Accuracy:", test_acc_wide_and_deep)
print("F1 Score:", f1_wide_and_deep)
print("ROC AUC Score:", roc_auc_wide_and_deep)

Wide and Deep Model:
Test Accuracy: 0.4449999928474426
F1 Score: 0.4331471905557104
ROC AUC Score: 0.8547627388888888


In [37]:
print("\nDeep Model:")
print("Test Accuracy:", test_acc)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)


Deep Model:
Test Accuracy: 0.4449000060558319
F1 Score: 0.43683983851015107
ROC AUC Score: 0.854163061111111
